In [1]:
import pandas as pd
import numpy as np
df = pd.read_pickle('T_combined_final_single_selector.pkl')


In [2]:
df['fps_blend_comp'] = (np.around(np.vstack(df.fps_blend_comp),decimals=5 )).tolist()

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Removing the polymer blends with missing thermal property values

In [4]:
df.dropna(subset = ["value"], inplace=True)
df.reset_index(drop = True, inplace=True)
# df.to_csv('T_dataset_processed_fox_fp.csv')


In [5]:
df.fillna(0, inplace=True)
# np.array(df.bdid.values, dtype=str)

# Changing the miscibility column to integer from boolean

In [6]:
df['miscibility'] = df.miscibility.values.astype(int)

In [7]:
df

,selector,bdid,value,prop,type,c1,c2,miscibility,fps_blend_comp,val_1,val_2
0,"[0, 1, 0]",547,238.15,Tg,Blend,0.80,0.20,1,"[0.20375, 0.85296, 0.12903, 0.27517, 0.25, 0.4...",238.15,238.15
1,"[0, 1, 0]",547,248.15,Tg,Blend,0.50,0.50,1,"[0.26403, 0.66891, 0.07692, 0.19685, 0.14286, ...",248.15,248.15
2,"[0, 1, 0]",547,249.15,Tg,Blend,0.70,0.30,1,"[0.2232, 0.78726, 0.11111, 0.24795, 0.21212, 0...",249.15,249.15
3,"[0, 1, 0]",547,338.15,Tg,Blend,0.50,0.50,1,"[0.26403, 0.66891, 0.07692, 0.19685, 0.14286, ...",338.15,338.15
4,"[0, 1, 0]",547,342.15,Tg,Blend,0.80,0.20,1,"[0.20375, 0.85296, 0.12903, 0.27517, 0.25, 0.4...",342.15,342.15
...,...,...,...,...,...,...,...,...,...,...,...
23389,"[0, 0, 2]",728,626.15,Tm,Blend,0.30,0.70,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02493, 0...",601.15,626.15
23390,"[0, 0, 2]",728,627.15,Tm,Blend,0.85,0.15,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07402, 0...",601.15,627.15
23391,"[0, 0, 2]",728,627.15,Tm,Blend,0.98,0.02,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08632, 0...",601.15,627.15
23392,"[0, 0, 2]",728,627.15,Tm,Blend,0.95,0.05,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08346, 0...",602.15,627.15


In [8]:
df['composition_ratio_of_1st_coplymer'] = [[100,0,0]]*df.shape[0]

# Preparing the dataset for training using tensorflow

In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
# import pandas as pd
# import tensorflow as tf
tf.random.set_seed(123)


df = df.sample(frac=1, random_state=123)

target_property_scaler = MinMaxScaler

batch_size = 200

df['scaled_value'] = df['value']
skf = StratifiedKFold(n_splits=5)

props = df.prop.unique().tolist()

datasets = []
def get_dataset(df):
    fps_blends = np.stack(df.fps_blend_comp.values).astype(np.float32)

    selector = np.stack(df.selector.values).astype(np.float32)
    cp_c1 = np.stack(df.composition_ratio_of_1st_coplymer.values).astype(np.float32)

    target = df.scaled_value.values.astype(np.float32)[:, np.newaxis]
    bdid = np.stack(df.bdid.values).astype(np.str)[:, np.newaxis]
    
    dataset = tf.data.Dataset.from_tensor_slices(({'selector': selector, 'fps_blends': fps_blends, 'prop': df.prop, 'c1': df.c1[..., None], 'cp_c1':cp_c1, 'miscibility':df.miscibility[..., None] ,'type': df.type[..., None], 'bdid':bdid}, target))

    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


training_df, test_df = train_test_split(df, test_size=0.2, stratify=df.prop, random_state=123)
training_df, test_df = training_df.copy(), test_df.copy()

for train_index, val_index in skf.split(training_df, training_df.prop):
    
    train_df = df.iloc[train_index].copy()
    val_df = df.iloc[val_index].copy()
    
    # scale target values
    property_scaler = {}
    for prop in props:
        property_scaler[prop] = target_property_scaler()

        # train
        cond = train_df[train_df.prop == prop].index
        val = train_df.loc[cond, ['scaled_value']]
        train_df.loc[cond, ['scaled_value']] = property_scaler[prop].fit_transform(val)

        # val
        cond = val_df[val_df.prop == prop].index
        val = val_df.loc[cond, ['scaled_value']]
        val_df.loc[cond, ['scaled_value']] = property_scaler[prop].transform(val)

    datasets.append({'train': get_dataset(train_df), 'val': get_dataset(val_df), 'property_scaler': property_scaler})

# Create final dataset for ensemble
property_scaler_final = {}

for prop in props:
    property_scaler_final[prop] = target_property_scaler()
    
    # train
    cond = training_df[training_df.prop == prop].index
    val = training_df.loc[cond, ['scaled_value']]
    training_df.loc[cond, ['scaled_value']] = property_scaler_final[prop].fit_transform(val)

    # val
    cond = test_df[test_df.prop == prop].index
    val = test_df.loc[cond, ['scaled_value']]
    test_df.loc[cond, ['scaled_value']] = property_scaler_final[prop].transform(val)
    
datasets_final = {'train': get_dataset(training_df), 'test': get_dataset(test_df), 'property_scaler': property_scaler_final}



In [12]:
datasets_final

{'train': <PrefetchDataset element_spec=({'selector': TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), 'fps_blends': TensorSpec(shape=(None, 872), dtype=tf.float32, name=None), 'prop': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'c1': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'cp_c1': TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), 'miscibility': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), 'type': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'bdid': TensorSpec(shape=(None, 1), dtype=tf.string, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>,
 'test': <PrefetchDataset element_spec=({'selector': TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), 'fps_blends': TensorSpec(shape=(None, 872), dtype=tf.float32, name=None), 'prop': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'c1': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'cp_c1': TensorSpec(shape=(None, 3), dtype=tf.

# Defining the model

In [ ]:
import tensorflow.keras as tfk
import tensorflow as tf
from datetime import datetime 
from tensorflow.python.keras.engine import data_adapter
from keras_tuner import HyperModel
from keras_tuner.tuners import Hyperband, RandomSearch
import tensorflow_addons as tfa


df.fps_blend_comp.to_pickle('pkl/fingerprints_prediction_final.pkl')
class MetaModel(tfk.Model):
    def __init__(self, hp): #, scale):
        super().__init__()
        self.base_models = []
        # self.scaler_type = scale
        for num in range(5):
            model = tf.keras.models.load_model(f'models/cv_final_model/{num}')
            model.trainable = False
            self.base_models.append(model)
        
        self.my_layers = []
        
        for i in range(hp.Int('num_layers', 1, 2)): 
            new_step = [               
            tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=64,
                                            max_value=544,
                                            step=64),),
            
            tf.keras.layers.PReLU(),
            tf.keras.layers.Dropout(hp.Float(
                'dropout_' + str(i),
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )),]
            self.my_layers.append(new_step)
            
        self.my_layers.append([tf.keras.layers.Dense(1)])

    def call(self, inputs, training=None): 
        
        # drop prop
        if 'prop' in inputs:
            del inputs['prop']
        x = []
        for base in self.base_models:
            if training:
                res = base.call(inputs, training)
            else:
                res = base.call(inputs)
            x.append(res)
        x = tf.concat(x, -1)
        
        for num, layer_step in enumerate(self.my_layers):
            for layer in layer_step:
                x = layer(x)

        return x
    
    def predict_step(self, data):
        data = data_adapter.expand_1d(data)
        x, _, _ = data_adapter.unpack_x_y_sample_weight(data)
        # drop prop here
        prop = x['prop']
        # bdid = x['bdid']
        type_poly = x['type']
        selector = x['selector']
        bdid = x['bdid']
        c1 = x['c1']
        cp_c1 = x['cp_c1']
        miscibility = x['miscibility']
        del x['prop']
        # del x['bdid']
        # del x['selector']
        return self(x, training=True), data[-1], prop, type_poly, bdid, miscibility, c1, cp_c1, selector

    
    @tf.function(input_signature=[])
    def fp_order(self):
        fp_order = tf.convert_to_tensor(list(pd.read_pickle('pkl/fingerprints_prediction_final.pkl').columns))
        return fp_order
        
    
def build_model(hp):
    model = MetaModel(hp)
    opt = tf.keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-3]))
    opt = tfa.optimizers.SWA(opt)

    model.compile(
        optimizer=opt,
        loss='mse',)
    return model

# Training and optimizing the hyperparameters

In [ ]:
import IPython
from sklearn.metrics import mean_squared_error, r2_score
results, property_metric = [], []


tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=300,
    # hyperband_iterations=3,
    seed=10,
    directory=f'hyperparameter_search_meta_learner_final',
    project_name='fold_0',
    )

reduce_lr = tfk.callbacks.ReduceLROnPlateau(
    factor=0.9,
    monitor="val_loss",
    verbose=1,
)

earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=40)

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

# Create an instance of the model
tuner.search(datasets_final['test'],
            epochs=300,
            validation_data=datasets_final['train'],
            callbacks=[earlystop, reduce_lr, ClearTrainingOutput()],
            verbose=0
            )

# Post processing
best_values = tuner.get_best_hyperparameters()[0].values

best_model = tuner.get_best_models(1)[0]
res = np.concatenate(best_model.predict(datasets_final['train']), -1)
# best_model.save(f'models/meta_model_{what}/', include_optimizer=False)


_df = pd.DataFrame(res, columns=['pred', 'target', 'prop', 'type_poly', 'bdid', 'miscibility','c1', 'cp_c1_0', 'cp_c1_1', 'cp_c1_3', 'selector_0', 'selector_1', 'selector_2'])
_df['prop'] = _df.prop.apply(lambda x: x.decode('utf-8'))
# _df['bdid'] = _df.prop.apply(lambda x: x.decode('utf-8'))

props = _df.prop.unique()

property_scaler = datasets_final['property_scaler']
for prop in props:

    cond = _df[_df.prop == prop].index
    rmse_scaled = mean_squared_error(_df.loc[cond, ['target']], _df.loc[cond, ['pred']], squared=False)
    r2_scaled = r2_score(_df.loc[cond, ['target']], _df.loc[cond, ['pred']])
    
    _df.loc[cond, ['pred']] = property_scaler[prop].inverse_transform(_df.loc[cond, ['pred']])
    _df.loc[cond, ['target']] = property_scaler[prop].inverse_transform(_df.loc[cond, ['target']])
    
    rmse = mean_squared_error(_df.loc[cond, ['target']], _df.loc[cond, ['pred']], squared=False)
    r2 = r2_score(_df.loc[cond, ['target']], _df.loc[cond, ['pred']])
    property_metric.append({'name': f'meta_learner', 'prop': prop, 'rmse': rmse, 'r2':r2, 'rmse_scaled': rmse_scaled, 'r2_scaled': r2_scaled})
    
# No scaling back
rmse = mean_squared_error(res[:,0], res[:,1], squared=False)
r2 = r2_score(res[:,0], res[:,1])

results.append({'name': f'meta_learner', 'r2': r2, 'rmse':rmse})

pd.DataFrame(results)


In [16]:
best_values

{'num_layers': 2,
 'units_0': 320,
 'dropout_0': 0.0,
 'learning_rate': 0.001,
 'units_1': 384,
 'dropout_1': 0.25,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 34,
 'tuner/bracket': 4,
 'tuner/round': 3,
 'tuner/trial_id': 'fa11f84b740e4ec853b33c7756e77695'}

In [46]:
pd.DataFrame(property_metric)

,Tg,Td,Tm
rmse,26.271093,44.638685,23.287833
mae,18.618491,31.086444,16.986242
r2,0.862456,0.822291,0.891076
rmse_scaled,0.033129,0.050602,0.036114
r2_scaled,0.862456,0.822291,0.891076
